In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("csv_review_nlv.csv")

In [7]:
data.head()

,text,date,cool,funny,stars,business_id,useful,user_id,review_id
0,Had the Tonkotsu ramen for lunch upon the reco...,2017-05-02 20:37:33 UTC,0,0,4,-DnaKAs2oK3rXfrjSvn9ew,0,70p7ZC7JNkAn4YHJL2oLqQ,SvrM4R0Sn1085JldYArCwA
1,"Service was quick, food was excellent am glad ...",2015-10-09 01:30:39 UTC,0,0,5,-DnaKAs2oK3rXfrjSvn9ew,0,vRhxHrWwHpPDbzzKsCKIQw,6p-MEBAuZ4dhOA1k24vgdw
2,"I actually think that the place is a 3.5, so, ...",2015-06-07 00:33:50 UTC,0,0,4,-DnaKAs2oK3rXfrjSvn9ew,0,Bxf1jhd66YvcI0GFTIh3yQ,5P--fn1aEuuzsRiIgRcfCA
3,I eat alot of ramen and alot of curry.. but i ...,2017-07-22 20:35:05 UTC,0,0,5,-DnaKAs2oK3rXfrjSvn9ew,0,eRwwZ7CjgDuywdNZ_mtgKQ,XJLHswUbJYbzj9CmHpIsSw
4,Stopped in on a whim with my husband. Our stom...,2016-01-03 02:22:19 UTC,0,0,5,-DnaKAs2oK3rXfrjSvn9ew,0,PBSkpQxdUzx-4LyvEgX1KQ,FieX2hOMO2R0z5hj59sXCg


In [14]:
from gensim import corpora, models
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\7353228\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [54]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import os
from gensim.models.wrappers import LdaMallet



# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

import en_core_web_sm

In [31]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])


In [19]:
data.head()

,text,date,cool,funny,stars,business_id,useful,user_id,review_id
0,Had the Tonkotsu ramen for lunch upon the reco...,2017-05-02 20:37:33 UTC,0,0,4,-DnaKAs2oK3rXfrjSvn9ew,0,70p7ZC7JNkAn4YHJL2oLqQ,SvrM4R0Sn1085JldYArCwA
1,"Service was quick, food was excellent am glad ...",2015-10-09 01:30:39 UTC,0,0,5,-DnaKAs2oK3rXfrjSvn9ew,0,vRhxHrWwHpPDbzzKsCKIQw,6p-MEBAuZ4dhOA1k24vgdw
2,"I actually think that the place is a 3.5, so, ...",2015-06-07 00:33:50 UTC,0,0,4,-DnaKAs2oK3rXfrjSvn9ew,0,Bxf1jhd66YvcI0GFTIh3yQ,5P--fn1aEuuzsRiIgRcfCA
3,I eat alot of ramen and alot of curry.. but i ...,2017-07-22 20:35:05 UTC,0,0,5,-DnaKAs2oK3rXfrjSvn9ew,0,eRwwZ7CjgDuywdNZ_mtgKQ,XJLHswUbJYbzj9CmHpIsSw
4,Stopped in on a whim with my husband. Our stom...,2016-01-03 02:22:19 UTC,0,0,5,-DnaKAs2oK3rXfrjSvn9ew,0,PBSkpQxdUzx-4LyvEgX1KQ,FieX2hOMO2R0z5hj59sXCg


In [21]:
reviews = data.text.values.tolist()

In [22]:
# Remove new line characters
reviews = [re.sub('\s+', ' ', sent) for sent in reviews]

In [23]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(reviews))

In [32]:
data_words[1]

['service',
 'was',
 'quick',
 'food',
 'was',
 'excellent',
 'am',
 'glad',
 'my',
 'wife',
 'and',
 'daughter',
 'saw',
 'this',
 'place',
 'and',
 'read',
 'the',
 'reviews',
 'before',
 'we',
 'went',
 'to',
 'the',
 'cannery',
 'buffet']

In [25]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['had', 'the', 'tonkotsu_ramen', 'for', 'lunch', 'upon', 'the', 'recommendation', 'from', 'co_worker', 'very', 'well', 'made', 'with', 'tasty', 'broth', 'the', 'slices', 'of', 'pork', 'just', 'melted', 'in', 'my', 'mouth', 'very', 'filling', 'and', 'tasty', 'have', 'returned', 'to', 'try', 'out', 'some', 'of', 'the', 'other', 'soups', 'update', 'the', 'vegetable', 'udon', 'noodle_soup', 'was', 'also', 'very', 'good', 'the', 'ambience', 'and', 'decor', 'need', 'some', 'work', 'but', 'the', 'service', 'is', 'very', 'courteous', 'and', 'the', 'soups', 'are', 'great']


In [26]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [38]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
#python3 -m spacy download en
nlp = en_core_web_sm.load()

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['tonkotsu_ramen', 'lunch', 'recommendation', 'well', 'make', 'tasty', 'broth', 'slice', 'pork', 'melt', 'mouth', 'fill', 'tasty', 'return', 'try', 'soup', 'update', 'vegetable', 'udon', 'noodle_soup', 'also', 'good', 'ambience', 'decor', 'need', 'work', 'service', 'courteous', 'soup', 'great']]


In [39]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 2), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1)]]


In [40]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('also', 1),
  ('ambience', 1),
  ('broth', 1),
  ('courteous', 1),
  ('decor', 1),
  ('fill', 1),
  ('good', 1),
  ('great', 1),
  ('lunch', 1),
  ('make', 1),
  ('melt', 1),
  ('mouth', 1),
  ('need', 1),
  ('noodle_soup', 1),
  ('pork', 1),
  ('recommendation', 1),
  ('return', 1),
  ('service', 1),
  ('slice', 1),
  ('soup', 2),
  ('tasty', 2),
  ('tonkotsu_ramen', 1),
  ('try', 1),
  ('udon', 1),
  ('update', 1),
  ('vegetable', 1),
  ('well', 1),
  ('work', 1)]]

In [41]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [42]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.410*"tell" + 0.134*"visit" + 0.099*"shop" + 0.068*"almost" + '
  '0.040*"often" + 0.037*"low" + 0.035*"face" + 0.033*"wonderful" + '
  '0.024*"word" + 0.021*"truly"'),
 (1,
  '0.098*"good" + 0.095*"place" + 0.063*"service" + 0.047*"great" + '
  '0.040*"really" + 0.039*"never" + 0.037*"price" + 0.037*"customer" + '
  '0.034*"nice" + 0.031*"experience"'),
 (2,
  '0.348*"work" + 0.087*"door" + 0.084*"box" + 0.077*"stay" + 0.072*"fill" + '
  '0.048*"update" + 0.047*"smell" + 0.046*"life" + 0.028*"opinion" + '
  '0.024*"mouth"'),
 (3,
  '0.250*"chicken" + 0.128*"table" + 0.073*"cook" + 0.069*"rice" + '
  '0.063*"plate" + 0.050*"egg" + 0.043*"quickly" + 0.040*"combo" + '
  '0.036*"onion" + 0.032*"green"'),
 (4,
  '0.089*"look" + 0.081*"see" + 0.068*"wait" + 0.062*"could" + 0.060*"review" '
  '+ 0.053*"still" + 0.041*"long" + 0.040*"check" + 0.039*"lot" + '
  '0.034*"next"'),
 (5,
  '0.343*"anyone" + 0.193*"complaint" + 0.155*"damn" + 0.076*"tiny" + '
  '0.031*"surround" + 0.000*"ca

In [43]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -17.100372484796043

Coherence Score:  0.26801387284061023


In [44]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\7353228\Anaconda\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.301321 -0.198901       1        1  49.931210
1     -0.274496 -0.168782       2        1  17.210304
4     -0.183654  0.395414       3        1   8.442111
18    -0.044600  0.059574       4        1   2.474253
0     -0.027894  0.042798       5        1   2.239444
19    -0.008437  0.025084       6        1   2.029770
2     -0.006372  0.023935       7        1   1.914967
14     0.012430  0.009913       8        1   1.738947
12    -0.001221  0.019791       9        1   1.725999
9      0.022930  0.002948      10        1   1.692318
11     0.018593  0.005765      11        1   1.644356
13     0.018273  0.005912      12        1   1.572550
15     0.027894 -0.000235      13        1   1.455491
3      0.027954 -0.000344      14        1   1.422832
17     0.061213 -0.018310      15        1   1.079244
10     0.111923 -0.038120      16        1   0.920877
6      0.105656 -0.036124      17        1   0.886872
7      0.084151 -0.028303      18        1   0.867320
16     0.178007 -0.051004      19        1   0.444321
5      0.178972 -0.051012      20        1   0.306813, topic_info=     Category          Freq       Term         Total  loglift  logprob
165   Default  18755.000000       tell  18755.000000  30.0000  30.0000
6     Default  34441.000000       good  34441.000000  29.0000  29.0000
35    Default  33324.000000      place  33324.000000  28.0000  28.0000
27    Default  13609.000000       work  13609.000000  27.0000  27.0000
110   Default   9637.000000        new   9637.000000  26.0000  26.0000
17    Default  22128.000000    service  22128.000000  25.0000  25.0000
187   Default  15321.000000       look  15321.000000  24.0000  24.0000
60    Default  52698.000000        get  52698.000000  23.0000  23.0000
454   Default   8198.000000        car   8198.000000  22.0000  22.0000
39    Default  13918.000000        see  13918.000000  21.0000  21.0000
106   Default   8290.000000      leave   8290.000000  20.0000  20.0000
647   Default   7791.000000       year   7791.000000  19.0000  19.0000
49    Default   7275.000000    chicken   7275.000000  18.0000  18.0000
371   Default  11803.000000       wait  11803.000000  17.0000  17.0000
7     Default  16585.000000      great  16585.000000  16.0000  16.0000
34    Default  41996.000000         go  41996.000000  15.0000  15.0000
384   Default   6096.000000  recommend   6096.000000  14.0000  14.0000
228   Default   6137.000000       menu   6137.000000  13.0000  13.0000
274   Default   6686.000000        fry   6686.000000  12.0000  12.0000
175   Default   7475.000000         do   7475.000000  11.0000  11.0000
97    Default  10677.000000      could  10677.000000  10.0000  10.0000
337   Default   5546.000000      offer   5546.000000   9.0000   9.0000
38    Default  10341.000000     review  10341.000000   8.0000   8.0000
55    Default   5433.000000      drink   5433.000000   7.0000   7.0000
145   Default  17544.000000     really  17544.000000   6.0000   6.0000
287   Default  13862.000000      never  13862.000000   5.0000   5.0000
761   Default   5851.000000        put   5851.000000   4.0000   4.0000
122   Default   5106.000000       stop   5106.000000   3.0000   3.0000
636   Default   5790.000000       open   5790.000000   2.0000   2.0000
415   Default   6120.000000      visit   6120.000000   1.0000   1.0000
...       ...           ...        ...           ...      ...      ...
5057  Topic20      0.037613      vegan      0.868835   2.6469 -12.0228
1676  Topic20      0.037612     refund      0.868825   2.6469 -12.0228
913   Topic20      0.037637       move      0.869598   2.6466 -12.0222
1341  Topic20      0.037631    company      0.869428   2.6467 -12.0223
1861  Topic20      0.037614      write      0.868909   2.6468 -12.0228
1250  Topic20      0.037615       pool      0.868937   2.6468 -12.0227
844   Topic20      0.037619      owner      0.869111   2.6467 -12.0226


In [55]:
os.environ['MALLET_HOME'] = 'C:\\users\\7353228\\Documents\\Data Science\\AMLI\\AMLI-Final-Project\\mallet-2.0.8'

In [57]:
mallet_path = 'C:\\users\\7353228\\Documents\\Data Science\\AMLI\\AMLI-Final-Project\\mallet-2.0.8\\bin\\mallet'
ldamallet = LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

CalledProcessError: Command 'C:\users\7353228\Documents\Data Science\AMLI\AMLI-Final-Project\mallet-2.0.8\bin\mallet import-file --preserve-case --keep-sequence --remove-stopwords --token-regex "\S+" --input C:\Users\7353228\AppData\Local\Temp\39f920_corpus.txt --output C:\Users\7353228\AppData\Local\Temp\39f920_corpus.mallet' returned non-zero exit status 1.

In [ ]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)